In [ ]:
!pip install simpletransformers


In [ ]:
%pip install wandb -q

In [ ]:
import numpy as np
import pandas as pd
import os
import re
import time
from gensim.models import Word2Vec
from gensim.models import FastText
#from gensim.models.wrappers import FastText
from tqdm import tqdm
from pprint import pprint
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt 
from google.colab import files
import io
from sklearn.metrics import classification_report
#%matplotlib inline

#uploaded = files.upload()
data_path = 'drive/My Drive/Work/data.txt'
f = open(data_path, "r")

#print(Sentences[2][1])
df = pd.DataFrame()
Sentences=[]
count=0
for s in f:
    a=0
    d=[]
    count+=1
    if ";" in s:
        '''if s.count(';') != 2:   Finding number of wrongly formatted lines
            print(count)'''
        b = s.split(";")
        b[2] = b[2].lower()
    #print(len(b[2]))
        for x in range(len(b[2])):
            if (b[2][x] == '.' or b[2][x] == '?' or b[2][x] == '!' or b[2][x] == '…'):
                d.append(b[2][a:x])
                a=x+1
        
        df.loc[count, "Name"] = b[1]
        
        df.loc[count, "text"] = b[2]


        for x in range(len(d)):
            sent = d[x].translate({ord(i): None for i in ',-.'})
            Sentences.append(sent.split()) # Need to remove non-alphabetical characters from the lists

#df.to_csv('DataTable.csv')
#print(Sentences)

df['Name'] = df['Name'].replace(['Hero', 'Varun', ' Varunu', '06-18:08:Varun', 'varun', ' Varun'], 'Varun')
df['Name'] = df['Name'].replace(['Balraju', 'Bala Raju', 'bala Raju', 'Bala raju', ' Bala Raju'], 'Balraju')
df['Name'] = df['Name'].replace(['Heroine', 'Jagadamba', 'J', 'Jagadhamba', 'Jagdamba', 'Jagadhmba', 'Jagdhamba', ' Jagadhamba', 'Jgadhamba', 'Jagadhama'], 'Jagadamba')
df['Name'] = df['Name'].replace(['krishna', 'Krshna', 'Krishana', 'Kroshna'], 'Krishna')
df['Name'] = df['Name'].replace(['Heroine father', 'Jagadamba father', 'J father', 'Guravayya', 'Guravayy', ' J father', 'Guravyya'], 'Guruvayya') 
df['Name'] = df['Name'].replace(['Heroine grandma', 'Jagadamba grandma', 'J Grand mother', 'Baamma', 'J Grand Mother', 'Grand Mother'], 'J grandma')
df['Name'] = df['Name'].replace(['Boss'], 'Ramesh')
df['Name'] = df['Name'].str.replace('J.s Grand Moyther', 'J grandma', regex=True)
df['Name'] = df['Name'].str.replace('J.s Grand Mother', 'J grandma', regex=True)
df['Name'] = df['Name'].str.replace('Jagadhamba.s Grand Mother', 'J grandma', regex=True)
df['Name'] = df['Name'].str.replace('J.sGrand Mother', 'J grandma', regex=True)
df['Name'] = df['Name'].str.replace('Heroin.s Grand Mother', 'J grandma', regex=True)
df['Name'] = df['Name'].str.replace('Hero-in.s Father', 'Guruvayya', regex=True)
df['Name'] = df['Name'].str.replace('Heroin.s Father', 'Guruvayya', regex=True)
#AllNames = df['Name'].unique().tolist()

#Dialogues_by_Speaker = df.groupby("Name")["Utterance"].count().nlargest(40)
#print(Dialogues_by_Speaker.head(40))

df.loc[:, 'labels'] = 6
df.loc[df['Name'] == 'Varun', 'labels'] = 0
df.loc[df['Name'] == 'Balraju', 'labels'] = 1
df.loc[df['Name'] == 'Jagadamba', 'labels'] = 2
df.loc[df['Name'] == 'Krishna', 'labels'] = 3
df.loc[df['Name'] == 'Guruvayya', 'labels'] = 4
df.loc[df['Name'] == 'Ramesh', 'labels'] = 5

df = df.drop(columns=['Name'])
print(df.head(20))

train_df=df.sample(frac=0.75,random_state=200) #random state is a seed value
train_df=train_df[train_df['labels'] != 6]
eval_df=df.drop(train_df.index).sample(frac=1.0)
eval_df=eval_df[eval_df['labels'] != 6]

print(train_df.head())
print(eval_df.head())
#pprint(AllNames)

In [ ]:
import wandb
wandb.login()

In [ ]:
from simpletransformers.classification import ClassificationModel

model = ClassificationModel('bert', 'bert-base-multilingual-cased', num_labels=6, args={'learning_rate':1e-5, 'num_train_epochs': 30, 'reprocess_input_data': True, 'overwrite_output_dir': True, 'save_model_every_epoch': False, 'save_steps': 5000})


In [ ]:
print(train_df.head())
print(eval_df.head())
model.train_model(train_df, eval_df=eval_df)




In [ ]:
from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
result, model_outputs, wrong_predictions =wmodel.eval_model(eval_df, f1=f1_multiclass, acc=accuracy_score)


In [ ]:
x_test = eval_df['text'].values
y_test = eval_df['labels'].values

predictions, raw_outputs = model.predict(x_test)
print(classification_report(y_test, predictions))

In [ ]:
# !zip -r /content/output.zip /content/outputs/
# files.download("/content/output.zip")

In [ ]:
print(result)